# Generate CSV file with percentages of unfeasibility for each sub group in the results CSV file

In [ ]:
using Serialization
using CSV
using DataFrames
using Plots

include("utils.jl")

In [ ]:
DM_l = [10, 15, 20, 25]
AM_l = [1, 2]

XM_l = [6, 9, 12, 15]            # max queue length 
YM_l = [10, 15, 20, 25]        # max buffer length before dropping calls

serM_nd_l = [5, 10, 15, 20, 25]        # max number of servers original model


df = CSV.read("results.csv", DataFrame);
df = filter(row -> row[:model_id] == "d_os" && row[:XM] != 3 && row[:YM] != 5 && row[:aM] != 3 && row[:dM] != 5, df);

In [ ]:
result_col = "cost"

DM_col  = "dM"
AM_col  = "aM"
XM_col  = "XM"
YM_col  = "YM"
serM_col= "serM"

function calc_mean_and_nan_prop(df::SubDataFrame, mean_col, nan_col)
    m = mean_col isa Symbol ? mean_col : Symbol(mean_col)
    n = nan_col  isa Symbol ? nan_col  : Symbol(nan_col)

    col_mean = df[!, m]
    is_nan_val(x) = x isa AbstractFloat && isnan(x)

    vals = [x for x in col_mean if !ismissing(x) && !is_nan_val(x)]
    mean_val = isempty(vals) ? missing : sum(vals) / length(vals)

    col_nan = df[!, n]
    total = nrow(df)
    count_nan = sum(x -> (!ismissing(x) && (is_nan_val(x) || (x isa AbstractString && x == "NaN"))), col_nan)
    prop_nan = total == 0 ? missing : count_nan / total

    return mean_val, prop_nan
end

vars_list = [DM_col, AM_col, XM_col, YM_col, serM_col]

xs = vars_list
for n_vars in 1:4
    idx = collect(1:n_vars)
    combos = []
    while true
        push!(combos, [xs[i] for i in idx])
        # find rightmost index that can be incremented
        j = n_vars
        while j >= 1 && idx[j] == 5 - n_vars + j
            j -= 1
        end
        j == 0 && break
        idx[j] += 1
        for k in (j+1):n_vars
            idx[k] = idx[k-1] + 1
        end
    end
    for combo_cols in combos 
        groups = groupby(df, combo_cols)
            for group in groups 
                group_name = "" 
                for col in combo_cols
                    group_name = group_name*col*" "*string(group[!, col][1])*" "
                end 
                mean, prop = calc_mean_and_nan_prop(group, "count", "cost")
                println(group_name, " ", mean, " ", prop)

                cs_df = DataFrame(group_name=group_name, mean_count=mean, prop_infeasible=prop)
                CSV.write("borders_copy.csv", cs_df, append=true)
            end
    end
end